In [ ]:


!pip install diffusers transformers accelerate scipy ftfy

import torch
from diffusers import StableDiffusionPipeline
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

# Function to evaluate the diffusion model
def evaluate_diffusion_model(prompt, num_inference_steps=50, guidance_scale=7.5):
    pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
    pipe = pipe.to("cuda")

    results = []
    # Assuming you have a list of reference images called 'reference_images'
    # Replace with your actual reference images
    reference_images = [Image.new("RGB", (512, 512), color = "red")] # Placeholder - replace with your actual images

    for ref_image in reference_images:
        image = pipe(prompt, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images[0]

        # Resize images for SSIM/PSNR if necessary
        image = image.resize(ref_image.size)

        # Convert images to numpy arrays for SSIM/PSNR calculations
        image_np = np.array(image)
        ref_image_np = np.array(ref_image)

        # Calculate SSIM and PSNR
        try:
          ssim_val = ssim(image_np, ref_image_np, channel_axis=2, data_range=image_np.max() - image_np.min())
          psnr_val = psnr(image_np, ref_image_np, data_range=image_np.max() - image_np.min())
        except ValueError as e:  # Handle potential ValueError from skimage.metrics
          print(f"Skipping image pair due to error: {e}")
          ssim_val = 0 # set to zero or handle appropriately
          psnr_val = 0
        results.append({'ssim': ssim_val, 'psnr': psnr_val})
    return results


# Example usage
prompt = "a photo of an astronaut riding a horse on mars"  # Replace with your prompt
num_inference_steps_list = [25, 50] # Example values
guidance_scale_list = [7.5, 10] # Example values

all_results = {}
for num_inference_steps in num_inference_steps_list:
    for guidance_scale in guidance_scale_list:
        print(f"Evaluating with num_inference_steps={num_inference_steps}, guidance_scale={guidance_scale}")
        results = evaluate_diffusion_model(prompt, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale)
        avg_ssim = np.mean([r['ssim'] for r in results])
        avg_psnr = np.mean([r['psnr'] for r in results])
        all_results[(num_inference_steps, guidance_scale)] = {'avg_ssim': avg_ssim, 'avg_psnr': avg_psnr}
        print(f"Average SSIM: {avg_ssim:.4f}, Average PSNR: {avg_psnr:.4f}")

print("Overall Results:", all_results)



   ---------------------------------------- 0.0/3.6 MB ? eta -:--:--
   -------- ------------------------------- 0.8/3.6 MB 8.3 MB/s eta 0:00:01
   -------------------- ------------------- 1.8/3.6 MB 4.6 MB/s eta 0:00:01
   -------------------------------- ------- 2.9/3.6 MB 4.3 MB/s eta 0:00:01
   -------------------------------------- - 3.4/3.6 MB 4.0 MB/s eta 0:00:01
   ---------------------------------------- 3.6/3.6 MB 3.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/10.4 MB 5.0 MB/s eta 0:00:02
   ------- -------------------------------- 1.8/10.4 MB 4.4 MB/s eta 0:00:02
   ---------- ----------------------------- 2.6/10.4 MB 4.1 MB/s eta 0:00:02
   ------------- -------------------------- 3.4/10.4 MB 4.1 MB/s eta 0:00:02
   ---------------- ----------------------- 4.2/10.4 MB 4.1 MB/s eta 0:00:02
   ------------------- -------------------- 5.0/10.4 MB 4.0 MB/s eta 0:00:02
   ----------------

C:\Users\likhitha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
